In this chapter, we will create a class to represent a multidimensional Vector class. Vector will behave like a standard Python immutable flat sequence. Its elements will be floats.
## `Vector`: A user-defined sequence type
### `Vector` Take #1: `Vector2d` Compatible

In [1]:
from array import array
import reprlib
import math

class Vector:
    typecode = 'd'
    def __init__(self, components):
        # 'd' means the elements are float
        self._components = array(self.typecode, components)
    def __iter__(self):
        return iter(self._components)
    def __repr__(self):
        # get a limited-length representation
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(self._components)
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    def __abs__(self):
        return math.hypot(*self)
    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        # handle slices of arrays without copying bytes.
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

### Protocol and Duck Typing
The **sequence protocol** in Python entails just the `__len__` and `__getitem__` methods. Any class `Spam` that implements those methods with the standard signature and semantics can be used anywhere a sequence is expected. 
### Vector Take #2: A Sliceable Sequence

In [2]:
from array import array
import reprlib
import math

class Vector:
    typecode = 'd'
    def __init__(self, components):
        # 'd' means the elements are float
        self._components = array(self.typecode, components)
    def __iter__(self):
        return iter(self._components)
    def __repr__(self):
        # get a limited-length representation
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(self._components)
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    def __abs__(self):
        return math.hypot(*self)
    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        # handle slices of arrays without copying bytes.
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    ##########################  NEW  ###
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        return self._components[index]
    ##########################  NEW  ###

v1 = Vector(range(7))
v1[1:5:2] # Vector is used like an sequence

array('d', [1.0, 3.0])

It would be better if a slice of a Vector was also a Vector instance and not a array.
### How Slicing works(not used in `Vector` but useful)
`[1:4]` is actually equivalent to `slice(1, 4, None)`

And `S.indices(len)` can fit the tricky slices (maybe with negative indices) within the `len`

In [3]:
slice(None, 10, 2).indices(5) # [:10:2] for a sequence with length 5 is actually 0,2,4, also [0:5:2]

(0, 5, 2)

### A Slice-Aware `__getitem__`

In [4]:
from array import array
import reprlib
import math
import operator

class Vector:
    typecode = 'd'
    def __init__(self, components):
        # 'd' means the elements are float
        self._components = array(self.typecode, components)
    def __iter__(self):
        return iter(self._components)
    def __repr__(self):
        # get a limited-length representation
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(self._components)
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    def __abs__(self):
        return math.hypot(*self)
    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        # handle slices of arrays without copying bytes.
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    def __len__(self):
        return len(self._components)
    ##########################  NEW  ###
    def __getitem__(self, index):
        if isinstance(index, slice):
            cls = type(self)
            return cls(self._components[index])
            # allow any of the numerous types of integers in NUmpy to be used as slices
            index = operator.index(index)
        return self._components[index]
    ##########################  NEW  ###

v1 = Vector(range(7))
v1[1:5:2]

Vector([1.0, 3.0])

### Vector Take #3: Dynamic Attribute Access
The special method `__getattr__` gives us the ability to access vector components by name (e.g., `v.x`, `v.y`) For example we want to access the first 4 components with `v.x, v.y, v.z, v.t`

The `__getattr__` method is invoked by the interpreter when attribute lookup fails. So when we assign a value directly to an attribute (althrough maybe not declared before), `__getattr__` method will not be invoked!!!

In [5]:
from array import array
import reprlib
import math
import operator

class Vector:
    typecode = 'd'
    ##########################  NEW  ###
    shortcut_names = 'xyzt'
    ##########################  NEW  ###
    def __init__(self, components):
        # 'd' means the elements are float
        self._components = array(self.typecode, components)
    ##########################  NEW  ###
    def __getattr__(self, name):
        cls = type(self)
        if (len(name) == 1): # one character
            pos = cls.shortcut_names.find(name) # if not found, return -1
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)
    ##########################  NEW  ###
    def __iter__(self):
        return iter(self._components)
    def __repr__(self):
        # get a limited-length representation
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(self._components)
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    def __abs__(self):
        return math.hypot(*self)
    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        # handle slices of arrays without copying bytes.
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        if isinstance(index, slice):
            cls = type(self)
            return cls(self._components[index])
            # allow any of the numerous types of integers in NUmpy to be used as slices
            index = operator.index(index)
        return self._components[index]

v1 = Vector(range(7))
v1.x = 5
v1, v1.x # NOT dynamic

(Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...]), 5)

`__setattr__` method makes it dynamic and avoids inconsistent behaviour

In [6]:
from array import array
import reprlib
import math
import operator

class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'
    def __init__(self, components):
        # 'd' means the elements are float
        self._components = array(self.typecode, components)
    def __getattr__(self, name):
        cls = type(self)
        if (len(name) == 1): # one character
            pos = cls.shortcut_names.find(name) # if not found, return -1
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)
    ##########################  NEW  ###
    def __setattr__(self, name, value):
        cls = type(self)
        if (len(name) == 1):
            if name in cls.shortcut_names:
                error = 'read-only attribute {attr_name!r}'
            else:
                error = '{cls_name!r} has no {attr_name!r}'
            msg = error.format(cls_name=cls.__name__, attr_name=name)
            raise AttributeError(msg)
        super().__setattr__(name, value)
    ##########################  NEW  ###
    def __iter__(self):
        return iter(self._components)
    def __repr__(self):
        # get a limited-length representation
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(self._components)
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    def __abs__(self):
        return math.hypot(*self)
    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        # handle slices of arrays without copying bytes.
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        if isinstance(index, slice):
            cls = type(self)
            return cls(self._components[index])
            # allow any of the numerous types of integers in NUmpy to be used as slices
            index = operator.index(index)
        return self._components[index]

v1 = Vector(range(7))
try:
    v1.x = 5
except AttributeError as e:
    print(e)

read-only attribute 'x'


### Vector Take #4: Hashing and a Faster ==
Use `functools.reduce` to apply the xor operator to the hashed of every component: `v[0] ^ v[1] ^...`

Let’s say we have a two-argument function `fn` and a list `lst`. When you call `reduce(fn, lst)`, `fn` will be applied to the first pair of elements—`fn(lst[0], lst[1])`—producing a first result, `r1`. Then `fn` is applied to `r1` and the next element—`fn(r1, lst[2])`—producing a second result, `r2`. Now `fn(r2, lst[3])` is called to produce `r3` … and so on until the last element, when a single result, `rN`, is returned.

In [7]:
import functools
print(1 * 2 * 3 * 4 * 5)

functools.reduce(lambda a, b: a * b, range(1, 6), 1) # the third argument is the initializer

120


120

In [8]:
from array import array
import reprlib
import math
import operator
import functools

class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'
    def __init__(self, components):
        # 'd' means the elements are float
        self._components = array(self.typecode, components)
    def __getattr__(self, name):
        cls = type(self)
        if (len(name) == 1): # one character
            pos = cls.shortcut_names.find(name) # if not found, return -1
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)
    def __setattr__(self, name, value):
        cls = type(self)
        if (len(name) == 1):
            if name in cls.shortcut_names:
                error = 'read-only attribute {attr_name!r}'
            else:
                error = '{cls_name!r} has no {attr_name!r}'
            msg = error.format(cls_name=cls.__name__, attr_name=name)
            raise AttributeError(msg)
        super().__setattr__(name, value)
    def __iter__(self):
        return iter(self._components)
    def __repr__(self):
        # get a limited-length representation
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(self._components)
    ##########################  NEW  ###
    def __eq__(self, other):
        return (len(self) == len(other) and all(a == b for a, b in zip(self, other))) # `zip` stops at the shortest operand 
    def __hash__(self):
        return functools.reduce(lambda a, b: hash(a) ^ hash(b), self, 0)
    ##########################  NEW  ###
    def __abs__(self):
        return math.hypot(*self)
    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        # handle slices of arrays without copying bytes.
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        if isinstance(index, slice):
            cls = type(self)
            return cls(self._components[index])
            # allow any of the numerous types of integers in NUmpy to be used as slices
            index = operator.index(index)
        return self._components[index]

v1 = Vector(range(7))
hash(v1)
v1 == Vector([0,1,2,3,4,5,6.0])

True